# 1-單純貝氏(Naive Bayes) (分類)

## [條件機率:](https://zh.wikipedia.org/wiki/%E6%9D%A1%E4%BB%B6%E6%A6%82%E7%8E%87) $ P(A|B)=\frac{P(A\cap{B})}{P(B)} $

## [貝氏定理](https://zh.wikipedia.org/wiki/%E8%B4%9D%E5%8F%B6%E6%96%AF%E5%AE%9A%E7%90%86) $ P(A|B)=\frac{P(B|A)P(A)}{P(B)} $

$ P(A|B)P(B)=P(A\cap{B})=P(B|A)P(A) $

# 2-詞向量
* 文章1: 我喜歡吃牛排，也喜歡吃雞排
* 文章2: 你喜歡閱讀和旅遊
* 切割1: 我/喜歡/吃/牛排/，/也/喜歡/吃/雞排
* 切割2: 你/喜歡/閱讀/和/旅遊

<table style="border:3px #cccccc solid;" border='1'>
    <tr>
        <td>特徵</td>
        <td>我</td>
        <td>喜歡</td>
        <td>吃</td>
        <td>牛排</td>
        <td>也</td>
        <td>雞排</td>
        <td>你</td>
        <td>閱讀</td>
        <td>和</td>
        <td>旅遊</td>
    </tr>
    <tr>
        <td>向量1</td>
        <td>1</td>
        <td>2</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>向量2</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
    </tr>
</table>

# 3-TF-IDF 方法
* tf: 這個詞出現在整篇文章的次數，出現越多次，代表這個詞越能代表整篇文章
* idf: 這個詞在我蒐集的全部文章出現過的文章數，出現越多次，代表這個詞是一個慣用詞，重要性下降。

# 4-資料集
* chinese_news: 原本的新聞
* chinese_trans: 翻譯成繁體的新聞
* chinese_tests: 我從原本的新聞每個分類擷取出 10 篇當作測試文章

In [49]:
import os
import jieba
import pandas as pd

In [52]:
def process_dirs(base_dir):
    df = pd.DataFrame(columns = ["類別","內容"])
    # os.walk 會走到檔案才停下來
    for dir_path, dir_names, file_names in os.walk(base_dir):
        for single_file in file_names:
            if not single_file.startswith("."):
                f = open(os.path.join(dir_path, single_file), "r", encoding = "utf-8")
                content = f.read()
                # 讀完黨以後做出第一步處理, 先把換行都去掉
                content = content.replace("\r", "").replace("\n", "")
                split_word = jieba.cut(content)
                # 分詞
                content = " ".join(split_word)
                s = pd.Series([dir_path.split("\\")[-1], content], index = ["類別", "內容"])
                df = df.append(s, ignore_index = True)
    df['類別'] = df['類別'].astype('category')
    return df

In [53]:
base_dir = "./data/chinese_news_trans"
test_dir = "./data/chinese_news_test"
df = process_dirs(base_dir)
df

,類別,內容
0,交通,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,交通,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,交通,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,交通,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,交通,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...
...,...,...
2632,體育,馬 玉芹 破 女子 ４ ０ ０ 米 跑 全國 青...
2633,體育,國際 奧委會 中國 臺 北委員 吳經國 訪問 北...
2634,體育,亞奧 理事 會 ３ ９ 個 成員 組織 全部 以...
2635,體育,世界 盃 乒乓球 團體賽 男子 團體 採用 新賽...


In [54]:
# 不重複顯示所有類別
print(df['類別'].cat.categories)

Index(['交通', '政治', '教育', '環境', '經濟', '藝術', '計算機', '軍事', '醫藥', '體育'], dtype='object')


In [55]:
# 找出對應的 index
print(df['類別'].cat.categories.get_loc('交通')) # 0
print(df['類別'].cat.categories.get_loc('政治')) # 1
print(df['類別'].cat.categories.get_loc('教育')) # 2
print(df['類別'].cat.categories.get_loc('環境')) # 3

0
1
2
3


In [56]:
# {類別名稱:index}的字典
saved_map = { cate:df['類別'].cat.categories.get_loc(cate) for cate in df['類別'].cat.categories }
saved_map

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [57]:
df['類別'] = df['類別'].cat.codes
df

,類別,內容
0,0,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,0,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,0,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,0,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,0,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...
...,...,...
2632,9,馬 玉芹 破 女子 ４ ０ ０ 米 跑 全國 青...
2633,9,國際 奧委會 中國 臺 北委員 吳經國 訪問 北...
2634,9,亞奧 理事 會 ３ ９ 個 成員 組織 全部 以...
2635,9,世界 盃 乒乓球 團體賽 男子 團體 採用 新賽...


In [58]:
# 把我們的測試資料讀取，並且使用剛剛存起來的 category 來 map
test_df = process_dirs(test_dir)
test_df

,類別,內容
0,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
1,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
2,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
3,交通,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...
4,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
...,...,...
96,體育,最 優秀 選手 無緣 亞運會 健美 賽 健美 在 亞洲 運動會 ...
97,體育,各國 記者 眼中 的 羽毛球 世錦賽 - - - - - - - - - - - -...
98,體育,友好 運動會 第五天 東道 主選手 大顯 神威 2001 年 09 月 03 日 02 ...
99,體育,不靠 技術 比運氣 第二 屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 舉行...


In [59]:
# 這邊必須使用剛剛存起來的字典來替換,因為如果直接使用 code 可能會發生沒對照到的事故
test_df['類別'] = test_df['類別'].replace(saved_map)
test_df

,類別,內容
0,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
1,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
2,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
3,0,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...
4,0,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
...,...,...
96,9,最 優秀 選手 無緣 亞運會 健美 賽 健美 在 亞洲 運動會 ...
97,9,各國 記者 眼中 的 羽毛球 世錦賽 - - - - - - - - - - - -...
98,9,友好 運動會 第五天 東道 主選手 大顯 神威 2001 年 09 月 03 日 02 ...
99,9,不靠 技術 比運氣 第二 屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 舉行...


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
# 注意一定要使用 fit_transform, 才會幫你轉換成詞向量
bag = vec.fit_transform(df['內容'])
print("總共維度:", len(vec.get_feature_names()))

總共維度: 96042


# 4-單純貝氏(Naive Bayes)

$ P(X_j|Y_k)=\frac{Count(X_j,Y_k)+1}{\sum_j^k(Count(X_j,Y_k)+1)} $

In [61]:
# 只要是文字, 我們通常就會選擇 MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB(alpha = 0.001).fit(bag, df['類別'])
test_bag = vec.transform(test_df['內容']) # 開始訓練
print("維度:", len(vec.get_feature_names()))

# 開始預測
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("Naive-Bayes 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

維度: 96042
預測: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Naive-Bayes 正確率:  100.0 %


# 5-K-近鄰演算法(KNN)

In [62]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, df['類別']) # 開始訓練

# 開始預測
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("kNN 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
kNN 正確率:  100.0 %
